In [1]:
import xarray as xr
import glob
from dask.diagnostics import ProgressBar
import numpy as np
import os

In [2]:
def reformat_time_dim(da):
    """
    Takes xarray dataarray and converts its time dim to an array of numpy
    
    Inputs:
    ----------------------------------------------------------------------
    da: Xarray dataarray(set maybe?) 
    
    Outputs:
    ----------------------------------------------------------------------
    da_reformat: Identical data to da, just different time dim format (numpy datetime46)
    """    
    # Extract the numerical values of year, month, day, hour, minute, & second
    years = da['time.year'].data
    months = da['time.month'].data
    days = da['time.day'].data
    hours = da['time.hour'].data
    minutes = da['time.minute'].data
    seconds = da['time.second'].data
    
    # Make list of tuples of (y, m, d, H, M, S)
    ymdHMS_list = []
    for i, _ in enumerate(da['time.year'].data):
        ymdHMS_list.append((years[i], months[i], days[i], hours[i], minutes[i], seconds[i]))

    # Convert tuples to datetime string and then make array of numpy datetimes
    dt_string_list = [f'{y:04d}-{m:02d}-{d:02d}T{H:02d}:{M:02d}:{S:02d}' 
                      for y, m, d, H, M, S in ymdHMS_list]
    dt_array = np.array(list(map(np.datetime64, dt_string_list)))
    da_reformatted = da.copy()
    da_reformatted = da_reformatted.assign_coords(dict(time=dt_array))
    
    return(da_reformatted)

In [3]:
region_slice = dict(lon=slice(30, 180), lat=slice(0, 60))

In [5]:
# model_list =  ['ACCESS-ESM1-5', 'CanESM5', 'IPSL-CM6A-LR', 'MIROC6', 'MRI-ESM2-0', 'HadGEM3-GC31-LL', 'CNRM-CM6-1']

model_list = [
    'ACCESS-CM2',
    # 'ACCESS-ESM1-5', 
    'BCC-CSM2-MR',
    # 'CanESM5', 
    # 'CNRM-CM6-1',
    # 'HadGEM3-GC31-LL', 
    # 'IPSL-CM6A-LR', 
    # 'MIROC6', 
    # 'MRI-ESM2-0', 
    'NorESM2-LM'
]

In [7]:
for model_name in model_list:
    print(model_name)
    print('Histnat')
    histnat_files = glob.glob(f'/home/disk/tc/pangulo/CMIP6/{model_name}/hist-nat/compiled_rlut_Amon_{model_name}_hist-nat_r1*.nc')
    histnat_ds = xr.open_mfdataset(histnat_files, chunks=dict(time=50, lat=50, lon=50)).sel(region_slice).rlut
    histnat_ds = histnat_ds.convert_calendar('noleap', align_on='year', use_cftime=True)
    histnat_ds = reformat_time_dim(histnat_ds).sel(time=slice('1/1/1950', None))
   
    with ProgressBar():
        print('Merging & Saving')
        file_name = f'/home/disk/p/pangulo/CATER-Project/HeatWave_Statistics/Data/{model_name}_hist-nat_rlut.nc'
        if os.path.isfile(file_name):
            os.remove(file_name)
        histnat_ds.compute().to_netcdf(file_name, mode='w')

ACCESS-CM2
Histnat
Merging & Saving
[#                                       ] | 3% Completed |  0.1s

/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  da_reformatted = da_reformatted.assign_coords(dict(time=dt_array))
/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by conv

[########################################] | 100% Completed |  1.2s
BCC-CSM2-MR
Histnat
Merging & Saving
[                                        ] | 0% Completed |  0.0s

/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  da_reformatted = da_reformatted.assign_coords(dict(time=dt_array))
/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by conv

[########################################] | 100% Completed |  3.3s
NorESM2-LM
Histnat
Merging & Saving
[                                        ] | 0% Completed |  0.1s

/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  da_reformatted = da_reformatted.assign_coords(dict(time=dt_array))
/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by conv

[########################################] | 100% Completed |  0.8s


In [8]:
for model_name in model_list:
    print(model_name)
    print('Historical')
    historical_files = glob.glob(f'/home/disk/tc/pangulo/CMIP6/{model_name}/historical/compiled_rlut_Amon_{model_name}_historical_r1*.nc')
    historical_ds = xr.open_mfdataset(historical_files, chunks=dict(time=50, lat=50, lon=50)).sel(region_slice).rlut
    historical_ds = historical_ds.convert_calendar('noleap', align_on='year', use_cftime=True)
    historical_ds = reformat_time_dim(historical_ds).sel(time=slice('1/1/1950', None))
    print('SSP')
    ssp_files = glob.glob(f'/home/disk/tc/pangulo/CMIP6/{model_name}/ssp245/compiled_rlut_Amon_{model_name}_ssp245_r1*.nc')
    ssp_ds = xr.open_mfdataset(ssp_files, chunks=dict(time=50, lat=50, lon=50)).sel(region_slice).rlut
    ssp_ds = ssp_ds.convert_calendar('noleap', align_on='year', use_cftime=True)
    ssp_ds = reformat_time_dim(ssp_ds).sel(time=slice(None, '1/1/2021'))
    with ProgressBar():
        print('Merging & Saving')
        ssp_extended_ds = xr.concat((historical_ds, ssp_ds), dim='time').sortby('time')
        file_name = f'/home/disk/p/pangulo/CATER-Project/HeatWave_Statistics/Data/{model_name}_ssp_extended_rlut.nc'
        if os.path.isfile(file_name):
            os.remove(file_name)
        ssp_extended_ds.compute().to_netcdf(file_name, mode='w')

ACCESS-CM2
Historical
SSP


/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  da_reformatted = da_reformatted.assign_coords(dict(time=dt_array))
/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by conv

Merging & Saving
[########################################] | 100% Completed |  1.2s
BCC-CSM2-MR
Historical


/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  da_reformatted = da_reformatted.assign_coords(dict(time=dt_array))
/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by conv

SSP
Merging & Saving
[#                                       ] | 2% Completed |  0.1s

/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  da_reformatted = da_reformatted.assign_coords(dict(time=dt_array))
/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by conv

[########################################] | 100% Completed |  3.2s
NorESM2-LM
Historical


/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  da_reformatted = da_reformatted.assign_coords(dict(time=dt_array))
/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by conv

SSP
Merging & Saving
[########################################] | 100% Completed |  0.7s


In [9]:
for model_name in model_list:
    print(model_name)
    print('Hist-GHG')
    histnat_files = glob.glob(f'/home/disk/tc/pangulo/CMIP6/{model_name}/hist-GHG/compiled_rlut_Amon_{model_name}_hist-GHG_r1*.nc')
    histnat_ds = xr.open_mfdataset(histnat_files, chunks=dict(time=50, lat=50, lon=50)).sel(region_slice).rlut
    histnat_ds = histnat_ds.convert_calendar('noleap', align_on='year', use_cftime=True)
    histnat_ds = reformat_time_dim(histnat_ds).sel(time=slice('1/1/1950', None))
   
    with ProgressBar():
        print('Merging & Saving')
        file_name = f'/home/disk/p/pangulo/CATER-Project/HeatWave_Statistics/Data/{model_name}_hist-GHG_rlut.nc'
        if os.path.isfile(file_name):
            os.remove(file_name)
        histnat_ds.compute().to_netcdf(file_name, mode='w')

ACCESS-CM2
Hist-GHG
Merging & Saving
[#                                       ] | 3% Completed |  0.1s

/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  da_reformatted = da_reformatted.assign_coords(dict(time=dt_array))
/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by conv

[########################################] | 100% Completed |  1.2s
BCC-CSM2-MR
Hist-GHG
Merging & Saving
[                                        ] | 0% Completed |  0.1s

/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  da_reformatted = da_reformatted.assign_coords(dict(time=dt_array))
/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by conv

[########################################] | 100% Completed |  3.3s
NorESM2-LM
Hist-GHG
Merging & Saving
[####                                    ] | 11% Completed |  0.1s

/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  da_reformatted = da_reformatted.assign_coords(dict(time=dt_array))
/tmp/ipykernel_945230/3294554851.py:31: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by conv

[########################################] | 100% Completed |  0.7s
